In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
init_load_flag = 1


### **Data Reading**

In [0]:
df = spark.sql("select * from databricks_cata.silver.customers_silver")
display(df)

### **Removing duplciates**

In [0]:
df= df.dropDuplicates(subset=['customer_id'])
display(df)

### dividing new vs old

In [0]:
if init_load_flag == 0:
    df_old = spark.sql("select DimCustomerKey , customer_id, create_date, update_date from databricks_cata.gold.DimCustomers")
else:
    df_old =spark.sql("select 0 DimCustomerKey , customer_id,0 create_date,0 update_date from databricks_cata.silver.customers_silver where 1=0")

In [0]:
df_old.display()

### Renaming Columns of df_old

In [0]:
df_old= df_old.withColumnRenamed("DimCustomerKey", "old_DimCustomerKey")\
    .withColumnRenamed("create_date", "old_create_date")\
    .withColumnRenamed("update_date", "old_update_date")\
    .withColumnRenamed("customer_id", "old_customer_id")
df_old.display()

### Applying join with old records

In [0]:
df_join = df.join(df_old, df.customer_id == df_old.old_customer_id, "left")

In [0]:
df_join.display()

### Separating new vs old records

In [0]:
df_new = df_join.filter(df_join.old_DimCustomerKey.isNull())

In [0]:
df_old = df_join.filter(df_join['old_DimCustomerKey'].isNotNull())

### Preparing DF OLd

In [0]:
df_old = df_old.drop("old_customer_id","old_update_date")

##renaming 
df_old = df_old.withColumnRenamed("old_DimCustomerKey", "DimCustomerKey")
df_old = df_old.withColumnRenamed("old_create_date", "create_date")
df_old = df_old.withColumn('create_date',to_timestamp(col("create_date")))

##Recrating "update data column"
df_old = df_old.withColumn("update_date",current_timestamp())


In [0]:
df_old.display()

###  Preparing DF_NEW

In [0]:
df_new.drop("old_DimCustomerKey","old_customer_id")

In [0]:
df_new = df_new.drop("old_DimCustomerKey","old_customer_id","old_update_date",'old_create_date')

##Recrating "update data column" and "current_date" column with current timestamp
df_new = df_new.withColumn("update_date",current_timestamp())
df_new = df_new.withColumn("create_date",current_timestamp())


In [0]:
df_new.display()

### Surrogate Key from 1

In [0]:
df_new= df_new.withColumn("DimCustomerKey", monotonically_increasing_id()+lit(1))

In [0]:
df_new.display()

### **Adding max surrogate key**

In [0]:
if init_load_flag ==1:
  max_surrogate_key = 0
else:
  df_maxsur = spark.sql("select max(DimCustomerKey) from databricks_cata.gold.DimCustomers")
  
  # converting df_maxsur to max_surrogate key variable
  max_surrogate_key = df_maxsur.collect()[0][max_surrogate_key]

In [0]:
df_new= df_new.withColumn("DimCustomerKey", lit(max_surrogate_key) + col("DimCustomerKey"))

### union of df_old and df_new


In [0]:
df_final = df_new.unionByName(df_old)

In [0]:
df_final.display()

### **SCD Type-1**

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("databricks_cata.gold.DimCustomers"):
    dlt_obj = DeltaTable.forPath(spark, "abfss://gold@sssadatabricksete.dfs.core.windows.net/DimCusotmers")
    dlt_obj.alias("trg").merge(df_final.alias("src"),"trg.DimCustomerKey = src.DimCustomerKey")\
          .whenMatchedUpdateAll()\
          .whenNotMatchedInsertAll()\
          .execute()

else :
      df_final.write.mode("overwrite")\
      .option("path","abfss://gold@sssadatabricksete.dfs.core.windows.net/DimCusotmers")\
      .format("delta")\
      .saveAsTable("databricks_cata.gold.DimCustomers")